In [3]:
import pandas as pd
import numpy as np
from ScoringPy import Processing
from pathlib import Path


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
current_path = Path.cwd()
RowData_path = f"{current_path.parent}\\Data\\RowData\\"


In [7]:
def process_applications():
    # Load the applications data from a Feather file located at RowData_path
    data = pd.read_feather(f"{RowData_path}applications.feather")

    # Calculate the complicity amount: difference between goods price and loan amount
    data["complicity_amount"] = data["goods_price"] - data["loan_amount"]

    # Calculate the complicity percentage: (complicity amount / loan amount) * 100
    data["complicity_percentage"] = (data["complicity_amount"] / data["loan_amount"]) * 100

    # Count the number of non-null external source scores for each row
    # The scores are from ext_source_score_1, ext_source_score_2, and ext_source_score_3
    data['ext_source_count'] = data[["ext_source_score_1", "ext_source_score_2", "ext_source_score_3"]].notnull().sum(axis=1)

    # Return the processed DataFrame
    return data

def process_contacts():
    # Load the contacts data from a Feather file located at RowData_path
    data = pd.read_feather(f"{RowData_path}contacts.feather")

    # Calculate the total number of mobile contact methods available for each row
    # This sums the values of mobile, emp_phone, work_phone, and phone columns
    data['contact_mobile_count'] = data[["mobile", "emp_phone", "work_phone", "phone"]].apply(lambda x: x.sum(), axis=1)

    # Return the processed DataFrame
    return data

def process_credit_info():

    def overdue_days():

        # Group by 'loan_id' and 'credit_status' to calculate the required stats
        loan_summary = data.groupby(['loan_id', 'credit_status'])['overdue_days'].agg(
            sum_overdue_days='sum',
            max_overdue_days='max'
        ).reset_index()

        # Calculate the total stats for each loan_id
        total_summary = data.groupby('loan_id')['overdue_days'].agg(
            sum_overdue_days='sum',
            max_overdue_days='max'
        ).reset_index()

        # Add a column for credit_status with value 'Total' for overall stats
        total_summary['credit_status'] = 'Total'

        # Concatenate loan_summary with total_summary
        final_summary = pd.concat([loan_summary, total_summary], axis=0).sort_values(by=['loan_id', 'credit_status']).reset_index(drop=True)

        # Pivot the data so that 'credit_status' values become columns
        pivoted_data = final_summary.pivot(index='loan_id', columns='credit_status',
                                           values=['sum_overdue_days', 'max_overdue_days'])

        # Flatten the multi-level columns
        pivoted_data.columns = [f"{stat}_{status}" for stat, status in pivoted_data.columns]

        # Reset the index for a cleaner DataFrame
        pivoted_data = pivoted_data.reset_index()

        return pivoted_data


    def loan_timings():
        # Grouping the data by 'loan_id' to calculate statistics related to 'days_since_credit'
        result = data.groupby('loan_id')['days_since_credit'].agg(
            days_since_first_loan_landed=lambda x: x.min(),  # Oldest loan date (minimum value of 'days_since_credit')
            days_since_last_loan_landed=lambda x: x.max(),   # Most recent loan date (maximum value of 'days_since_credit')
            avg_days_between_landing_loans=lambda x: x.sort_values().diff().mean()  # Average days between consecutive loans
        ).reset_index()  # Resetting index to make 'loan_id' a column in the result


        # Grouping data where 'days_since_end' is not null to calculate similar statistics
        result2 = data[data["days_since_end"].notna()].groupby('loan_id')['days_since_end'].agg(
            days_since_first_loan_ended=lambda x: x.min(),  # Oldest loan end date (minimum value of 'days_since_end')
            days_since_last_loan_ended=lambda x: x.max()    # Most recent loan end date (maximum value of 'days_since_end')
        ).reset_index()  # Resetting index for the second result set


        # Merging the two result sets on 'loan_id', keeping all rows from the first set
        combined_result = pd.merge(result, result2, on='loan_id', how='left')

        # Returning the final DataFrame containing the combined results
        return combined_result


    def historical_loans_count():
        # Group by 'loan_id' and 'credit_status', then count occurrences
        grouped_data = data.groupby(['loan_id', 'credit_status']).size().reset_index(name='count')

        # Calculate the total count for each 'loan_id'
        grouped_data['total_count'] = grouped_data.groupby('loan_id')['count'].transform('sum')

        # Pivot the data to transform 'credit_status' values into columns
        pivoted_data = grouped_data.pivot(index='loan_id', columns='credit_status', values='count').reset_index()

        # Add the total_count column
        pivoted_data['total_count'] = grouped_data.drop_duplicates(subset='loan_id')['total_count'].values

        # Fill missing values with 0 (if any credit_status is missing for a loan_id)
        pivoted_data = pivoted_data.fillna(0)

        pivoted_data.rename(columns={'Active': 'Active_loans_count',
                                     'Bad debt': 'Bad_loans_count',
                                     'Closed': 'Closed_loans_count',
                                     'Sold': 'Sold_loans_count',
                                     "total_count":"Total_loans_count"}, inplace=True)

        # Display the result
        return pivoted_data


    def historical_loans_types():
        # Group by 'loan_id' and 'credit_status', then count occurrences
        grouped_data = data.groupby(['loan_id', 'credit_type']).size().reset_index(name='count')

        # Calculate the total count for each 'loan_id'
        grouped_data['total_count'] = grouped_data.groupby('loan_id')['count'].transform('sum')

        # Pivot the data to transform 'credit_status' values into columns
        pivoted_data = grouped_data.pivot(index='loan_id', columns='credit_type', values='count').reset_index()

        # Add the total_count column
        pivoted_data['total_count'] = grouped_data.drop_duplicates(subset='loan_id')['total_count'].values

        # Fill missing values with 0 (if any credit_status is missing for a loan_id)
        pivoted_data = pivoted_data.fillna(0)

        # Display the result
        return pivoted_data


    def calculate_loan_stats():
        # Predefined list of columns for which statistics will be calculated
        columns = ['credit_amount', 'debt_amount', 'credit_limit', 'overdue_amount','annuity_amount','prolong_count','max_overdue_amount']

        result = data[['loan_id']].drop_duplicates()  # Start with unique loan IDs


        for column in columns:
            # Grouping by 'loan_id' and calculating statistics for each column
            stats = data.groupby('loan_id')[column].agg(
                avg_value='mean',  # Average of the column
                max_value='max',   # Maximum value
                min_value=lambda x: x[x > 0].min() if (x > 0).any() else x.min()   # Minimum value

            ).reset_index()  # Resetting index to make 'loan_id' a regular column

            # Rename columns to indicate which column they belong to
            stats = stats.rename(columns={
                'avg_value': f'{column}_avg',
                'max_value': f'{column}_max',
                'min_value': f'{column}_min',
            })

            # Merge the current stats with the result DataFrame on 'loan_id'
            result = result.merge(stats, on='loan_id', how='left')

        return result




    # Load the data and create initial DataFrame with unique loan IDs
    data = pd.read_feather(f"{RowData_path}creditinfo.feather")
    result = data[['loan_id']].drop_duplicates()

    # Step 1: Calculate overdue days stats
    overdue_data = overdue_days()
    result = result.merge(overdue_data, on='loan_id', how='left')
    # Comment: Merging overdue statistics to include both sum and max overdue days for different credit statuses.

    # Step 2: Calculate loan timings
    loan_timing_data = loan_timings()
    result = result.merge(loan_timing_data, on='loan_id', how='left')
    # Comment: Adding loan timing data to get insights on first/last loan landed and ended.

    # Step 3: Calculate historical loan counts by status
    loan_count_data = historical_loans_count()
    result = result.merge(loan_count_data, on='loan_id', how='left')
    # Comment: Including the count of loans by their credit status (Active, Closed, Bad Debt, Sold).

    # Step 4: Calculate historical loan types
    loan_type_data = historical_loans_types()
    result = result.merge(loan_type_data, on='loan_id', how='left')
    # Comment: Including the count of different loan types per loan_id.

    # Step 5: Calculate loan financial statistics
    loan_stats_data = calculate_loan_stats()
    result = result.merge(loan_stats_data, on='loan_id', how='left')
    # Comment: Adding financial stats like average, max, and min for different financial attributes of loans.

    return result

def process_documentation():
    # Load the contacts data from a Feather file located at RowData_path
    data = pd.read_feather(f"{RowData_path}documentations.feather")
    
    # Sum the values across the doc columns for each row
    data['contact_mobile_count'] = data[[f"doc_{i}" for i in range(1, 21)] ].apply(lambda x: x.sum(), axis=1)
    
    return  data

def process_geographical():
    data = pd.read_feather(f"{RowData_path}geographical.feather")

    data["avg_geographical_rating"] = (data["region_rating"] + data["city_region_rating"])/2
    
    return data

def process_historical_applications():
    
    def previous_contract_types():

        # Group by 'loan_id' and 'contract_type', then count occurrences
        grouped_data = data.groupby(['loan_id', 'contract_type']).size().reset_index(name='count')
    
        # Pivot the data to transform 'contract_type' values into columns
        pivoted_data = grouped_data.pivot(index='loan_id', columns='contract_type', values='count').reset_index()
    
    
        return pivoted_data


    def previous_contract_status():
    
        # Group by 'loan_id' and 'contract_status', then count occurrences
        grouped_data = data.groupby(['loan_id', 'contract_status']).size().reset_index(name='count')
    
        # Calculate the total count for each 'loan_id'
        grouped_data['total_count'] = grouped_data.groupby('loan_id')['count'].transform('sum')
    
        # Pivot the data to transform 'contract_status' values into columns
        pivoted_data = grouped_data.pivot(index='loan_id', columns='contract_status', values='count').reset_index()
    
        # Add the total_count column
        pivoted_data['total_count'] = grouped_data.drop_duplicates(subset='loan_id')['total_count'].values
    
    
        pivoted_data["reject_rate"] =  (pivoted_data["Refused"] / pivoted_data["total_count"])*100
        pivoted_data["Cancel_rate"] =  (pivoted_data["Canceled"] / pivoted_data["total_count"])*100
        pivoted_data["Approve_rate"] =  (pivoted_data["Approved"] / pivoted_data["total_count"])*100
    
        return pivoted_data
    
    
    def previous_contract_amounts():
        # Group by 'loan_id' and 'contract_status', then sum and average amounts
        grouped_data = data.groupby(['loan_id', 'contract_status']).agg(
            amount_sum=('approved_amount', 'sum'),
            amount_avg=('approved_amount', 'mean')
        ).reset_index()
    
        # Pivot the data to transform 'contract_status' values into columns
        pivoted_sum = grouped_data.pivot(index='loan_id', columns='contract_status', values='amount_sum').add_prefix('sum_')
        pivoted_avg = grouped_data.pivot(index='loan_id', columns='contract_status', values='amount_avg').add_prefix('avg_')
    
        # Combine sum and avg pivot tables
        pivoted_data = pivoted_sum.join(pivoted_avg).reset_index()
    
    
        return pivoted_data



    # Load the data and create initial DataFrame with unique loan IDs
    data = pd.read_feather(f"{RowData_path}hisotrical_applications.feather")
    result = data[['loan_id']].drop_duplicates()

    # Step 1: Calculate overdue days stats
    overdue_data = previous_contract_types()
    result = result.merge(overdue_data, on='loan_id', how='left')
    # Comment: Merging overdue statistics to include both sum and max overdue days for different credit statuses.

    # Step 2: Calculate loan timings
    loan_timing_data = previous_contract_status()
    result = result.merge(loan_timing_data, on='loan_id', how='left')
    # Comment: Adding loan timing data to get insights on first/last loan landed and ended.

    # Step 3: Calculate historical loan counts by status
    loan_count_data = previous_contract_amounts()
    result = result.merge(loan_count_data, on='loan_id', how='left')
    # Comment: Including the count of loans by their credit status (Active, Closed, Bad Debt, Sold).


    return result

def process_personal():
    data = pd.read_feather(f"{RowData_path}personal.feather")
    
    return data

def process_residence():
    data = pd.read_feather(f"{RowData_path}residence.feather")

    return data

def process_credit_card_balance():
    data = pd.read_feather(f"{RowData_path}credit_card_balance.feather")
    
    # Overall history
    grouped_data_total = data.groupby(['loan_id']).agg(
        max_credit_limit=('credit_limit', 'max'),
        total_atm_draw_amount=('atm_draw_amount', 'sum'),
        total_draw_amount=('draw_amount', 'sum'),
        total_other_draw_amount=('other_draw_amount', 'sum'),
        total_pos_draw_amount=('pos_draw_amount', 'sum'),
        total_atm_draw_count=('atm_draw_count', 'sum'),
        total_other_draw_count=('other_draw_count', 'sum'),
        total_pos_draw_count=('pos_draw_count', 'sum'),
        total_paid_installments=('paid_installments', 'sum'),
        max_dpd=('other_draw_count', 'max'),
    ).reset_index()

    # Last 12 months
    grouped_data_last_12_months = data[data["balance_month"] >= -12].groupby(['loan_id']).agg(
        max_credit_limit=('credit_limit', 'max'),
        total_atm_draw_amount=('atm_draw_amount', 'sum'),
        total_draw_amount=('draw_amount', 'sum'),
        total_other_draw_amount=('other_draw_amount', 'sum'),
        total_pos_draw_amount=('pos_draw_amount', 'sum'),
        total_atm_draw_count=('atm_draw_count', 'sum'),
        total_other_draw_count=('other_draw_count', 'sum'),
        total_pos_draw_count=('pos_draw_count', 'sum'),
        total_paid_installments=('paid_installments', 'sum'),
        max_dpd=('other_draw_count', 'max'),
    ).reset_index()

    # Last 6 months
    grouped_data_last_6_months = data[data["balance_month"] >= -6].groupby(['loan_id']).agg(
        max_credit_limit=('credit_limit', 'max'),
        total_atm_draw_amount=('atm_draw_amount', 'sum'),
        total_draw_amount=('draw_amount', 'sum'),
        total_other_draw_amount=('other_draw_amount', 'sum'),
        total_pos_draw_amount=('pos_draw_amount', 'sum'),
        total_atm_draw_count=('atm_draw_count', 'sum'),
        total_other_draw_count=('other_draw_count', 'sum'),
        total_pos_draw_count=('pos_draw_count', 'sum'),
        total_paid_installments=('paid_installments', 'sum'),
        max_dpd=('other_draw_count', 'max'),
    ).reset_index()


    # Rename columns for clarity before merging
    grouped_data_total = grouped_data_total.rename(
        columns={col: f"{col}_total" for col in grouped_data_total.columns if col != 'loan_id'}
    )

    grouped_data_last_12_months = grouped_data_last_12_months.rename(
        columns={col: f"{col}_last_12m" for col in grouped_data_last_12_months.columns if col != 'loan_id'}
    )

    grouped_data_last_6_months = grouped_data_last_6_months.rename(
        columns={col: f"{col}_last_6m" for col in grouped_data_last_6_months.columns if col != 'loan_id'}
    )

    # Merge overall data with last 12 months data
    combined_data = grouped_data_total.merge(
        grouped_data_last_12_months,
        on='loan_id',
        how='left'
    )

    # Merge the result with last 6 months data
    combined_data = combined_data.merge(
        grouped_data_last_6_months,
        on='loan_id',
        how='left'
    )

    combined_data["have_credit_card"] = "Yes"

    return combined_data

def process_pos_cash_balance():
    data = pd.read_feather(f"{RowData_path}pos_cash_balance.feather")

    filtered_data = data[data["balance_month"] == -1].dropna()

    grouped_data_with_max = filtered_data.groupby('loan_id').agg(
        sum_future_installments = ('future_installments', 'sum'),
        max_future_installments = ('future_installments', 'sum'),
        max_dpd= ('dpd', 'sum'),
        max_dpd_tolerance= ('dpd_tolerance', 'sum'),
        sum_dpd = ('dpd', 'sum'),
        sum_dpd_tolerance= ('dpd_tolerance', 'sum'),
    ).reset_index()

    return grouped_data_with_max

In [8]:
applications = process_applications()
contacts = process_contacts()
credit_info = process_credit_info()
documentation = process_documentation()
geographical = process_geographical()
historical_applications = process_historical_applications()
personal = process_personal()
residence = process_residence()
credit_card_balance = process_credit_card_balance()
pos_cash_balance = process_pos_cash_balance()

In [9]:
dfs = [
    contacts, credit_info, documentation, geographical,
    historical_applications, personal, residence,
    credit_card_balance, pos_cash_balance
]

# Start with the applications dataframe
merged_df = applications

# Perform successive left joins
for df in dfs:
    merged_df = merged_df.merge(df, on='loan_id', how='left')

In [11]:
merged_df.to_feather(RowData_path + "ML_data.feather")